<!--NAVIGATION-->
< [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [Contents](Index.ipynb) | [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) >

# NumPy数组的计算：通用函数

为什么NumPy在Python数据科学世界中如此重要？简单地说：NumPy提供了一个简单灵活的接口来优化数据数组的计算。

NumPy数组的计算有时非常快，有时也非常慢。使NumPy变快的关键是利用**向量化操作**，通常在NumPy的**通用函数（ufunc）**中实现（向量化操作也有其他实现方式，比如以后将介绍的**广播**）。

通用函数可以提高数组元素的重复计算的效率。本节将介绍NumPy包中常用且有用的数学通用函数。

## 缓慢的循环

Python的相对缓慢通常出现在很多小操作需要不断重复的时候，比如对数组的每个元素做循环操作时。一部分原因是python的动态性和解释性——数据类型灵活的特性决定了序列操作不能像C语言和Fortran语言一样被编译成有效的机器码。

举例：假设我们想计算数组每个元素的倒数，使用循环操作来解决问题时，代码如下：

In [63]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

这种实现方式可能对于有C语言或Java背景的人来说非常自然，但是如果测试一个很大量的输入数据运行上述代码的时间，这一操作将非常耗时！

下面我们来测试如果执行上述操作一百万次，花费的时间是多少。

In [64]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)#%timeit是ipython里的魔法命令

7.39 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


（我的电脑上）7秒多！在手机都以 Giga-FLOPS（即每秒十亿次浮点运算）为单位计算处理速度时，上面的处理结果所花费的时间确实是不合时宜的慢！

事实上，这里的处理瓶颈并不是运算本身，而是Python在每次循环时必须做数据类型的检查和函数的调度。每次进行倒数运算时，Python首先检查对象的类型，并且动态查找可以使用该数据类型的正确函数。如果我们在编译代码时进行这样的操作，那么就能在代码执行之前知晓类型的声明，结果的计算也会更加有效率。

## 通用函数（ufunc）介绍

NumPy为很多类型的操作提供了非常方便的、静态类型的、可编译程序的接口，也被称作**向量操作**。

当你对数组执行操作时，这个操作将会被用于数组中的每一个元素。这种向量方法被用于将循环推送至NumPy之下的编译层，这样会取得更快的执行效率。

比较以下两个结果：

In [94]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


如果计算一个较大数组的运行时间，可以看到它的完成时间比Python循环花费的时间更短：

In [67]:
%timeit (1.0 / big_array)

13.8 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


只有13毫秒多，时间仅为约1/500。

NumPy 中的向量操作是通过**通用函数**实现的。通用函数的主要目的是对NumPy数组中的值执行更快的重复操作。它非常灵活！

NumPy的向量操作支持对两个数组进行运算：

In [68]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

NumPy的向量操作支持进行多维数组的运算：

In [69]:
x = np.arange(9).reshape((3, 3))
2 ** x #**是指数运算

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]], dtype=int32)

只要你看到Python脚本中有上述循环，就应该考虑能否用**向量方式**替换这个循环。

## 探索NumPy的通用函数

通用函数有两种存在形式：**一元通用函数（unary ufunc）**对单个输入操作，**二元通用函数（binary ufunc）**对两个输入操作。

我们将在以下的介绍中看到这两种类型的例子。

### 1. 数组的运算

NumPy 通用函数的使用方式非常自然，因为它用到了Python原生的算术运算符，标准的加、减、乘、除都可以使用：

In [70]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # 向下取整，又称地板除法

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


还有逻辑非、指数运算符（``**``）和模运算符（``%``）的一元通用函数：

In [71]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


你可以任意将这些算术运算符组合使用。当然，你得考虑这些运算符的优先级：

In [72]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

所有这些算术运算符都是 NumPy 内置函数的简单封装器，例如``+`` 运算符就是一个``add`` 函数的封装器：

In [73]:
np.add(x, 2)

array([2, 3, 4, 5])

下表列出了所有NumPy实现的算术运算符:

|运算符	     | 通用函数            |描述                                   |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |加法运算 (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |减法运算 (e.g., ``3 - 2 = 1``)         |
|``-``          |``np.negative``      |负数运算 (e.g., ``-2``)                |
|``*``          |``np.multiply``      |乘法运算 (e.g., ``2 * 3 = 6``)         |
|``/``          |``np.divide``        |除法运算 (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |地板除法运算 (e.g., ``3 // 2 = 1``)    |
|``**``         |``np.power``         |指数运算 (e.g., ``2 ** 3 = 8``)        |
|``%``          |``np.mod``           |模/余数 (e.g., ``9 % 4 = 1``)          |

### 2. 绝对值

NumPy能**理解**Python内置的绝对值函数。对应的NumPy通用函数是``np.absolute``，该函数也可以用别名``np.abs``来访问：

In [74]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

In [75]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [76]:
np.abs(x)

array([2, 1, 0, 1, 2])

这个通用函数也可以处理复数。当处理复数时，绝对值返回的是该复数的**幅度**:

In [77]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### 3. 三角函数

NumPy提供的另一个好用的通用函数是三角函数。首先定义一个角度数组：

In [78]:
theta = np.linspace(0, np.pi, 3) #numpy.linspace()是等差数列函数，返回在指定范围内的均匀间隔的数字组成的数组
theta

array([0.        , 1.57079633, 3.14159265])

现在可以对这些值进行一些三角函数计算：

In [79]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


这些值是在机器精度内计算的，所以有些应该是0的值并没有精确到0。逆三角函数同样可以使用：

In [80]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### 4. 指数和对数

NumPy中另一个常用的运算通用函数是指数运算：

In [81]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x)) 
print("2^x   =", np.exp2(x)) 
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


指数运算的逆运算，即对数运算也是可用的。

最基本的``np.log``给出的是以自然数为底数的对数。如果你希望计算以2为底数或者以10为底数的对数，可以按照如下示例处理：

In [82]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


还有一些特殊的版本，对于非常小的输入值可以保持较好的精度：

In [83]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


当``x``的值很小时，以上函数给出的值比``np.log`` 或 ``np.exp``的计算更精确。

### 5.专用的通用函数

除了以上介绍到的，NumPy 还提供了很多通用函数，包括双曲三角函数、比特位运算、比较运算符、弧度转化为角度的运算、取整和求余运算，等等。

``scipy.special``提供了更加专业、更加晦涩少用的数学计算，下例展示了在统计学中用到的函数：

In [84]:
from scipy import special

In [85]:
# Gamma函数（广义阶乘， generalized factorials）和相关函数
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [86]:
# 误差函数（高斯积分）
# 它的实现和它的逆实现
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


NumPy 和 ``scipy.special``中提供了大量的通用函数，这些包可以通过搜索“gamma function python”得到。

## 高级的通用函数特性

下面将介绍一些通用函数的**特殊性质**。

### 1. 指定输出

在进行大量运算时，有时候**指定**一个用于存放运算结果的数组是非常有用的。

不同于创建临时数组，你可以用这个特性将计算结果直接写入到你期望的存储位置。

所有的通用函数都可以通过``out``参数来指定计算结果的存放位置：

In [87]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y) # multiply：数组和矩阵对应位置相乘，输出与相乘数组/矩阵的大小一致
print(y)

[ 0. 10. 20. 30. 40.]


这个特性也可以被用作**数组视图**，例如可以将计算结果写入指定数组的每隔一个元素的位置：

In [88]:
y = np.zeros(10)
np.power(2, x, out=y[::2]) #numpy.power(x1, x2) 数组x1的元素分别求x2次方。x2可以是数字，也可以是数组，但是x1和x2的列数要相同。
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


如果这里写的是``y[::2] = 2 ** x``，那么结果将是创建一个临时数组，该数组存放的是``2 ** x``的结果，并且接下来会将这些值复制到``y``数组中。

对于上述例子中比较小的计算量来说，这两种方式的差别并不大。但是对于较大的数组，**通过慎重使用``out``参数将能够有效节约内存。**

### 2. 聚合

二元通用函数有些非常有趣的**聚合功能**，这些聚合可以直接在对象上计算。例如，如果我们希望用一个特定的运算``reduce``一个数组，那么可以用任何通用函数的``reduce``方法。一个``reduce``方法会对给定的元素和操作重复执行，直至得到单个的结果。

例如，对``add``通用函数调用``reduce``方法会返回数组中所有元素的和：

In [89]:
x = np.arange(1, 6)
np.add.reduce(x) #reduce()函数即为迭代函数。每一次迭代，都将上一次的迭代结果与下一个元素一同传入二元func函数中去执行。

15

同样，对``multiply``通用函数调用``reduce``方法会返回数组中所有元素的乘积：

In [90]:
np.multiply.reduce(x)

120

如果需要存储每次计算的中间结果，可以使用``accumulate``：

In [91]:
np.add.accumulate(x) 
#accumulate 方法和reduce方法类似,只是它返回的数组和输入的数组的shape相同,保存所有的中间计算结果

array([ 1,  3,  6, 10, 15], dtype=int32)

In [92]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120], dtype=int32)

请注意，在一些特殊情况中,NumPy提供了专用的函数(``np.sum``, ``np.prod``, ``np.cumsum``, ``np.cumprod``) ,它们也可以实现以上reduce的功能，这些函数将在下节中具体介绍。

### 3. 外积（Outer products）

任何通用函数都可以用``outer``方法获得两个不同输入数组所有元素对的函数运算结果。这意味着你可以用一行代码实现一个乘法表：

In [93]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

## 通用函数：更多的信息

有关通用函数的更多信息（包括可用的通用函数的完整列表）可以在 [NumPy](http://www.numpy.org)和[SciPy](http://www.scipy.org) 文档的网站找到。

<!--NAVIGATION-->
< [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) | [Contents](Index.ipynb) | [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) >